In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Harry Bridges Plaza (Ferry Building)の需要予測

In [4]:
trip1 = pd.read_csv('/home/kyohei/bike_share_data/201402_babs_open_data/201402_trip_data.csv')
trip2 = pd.read_csv('/home/kyohei/bike_share_data/201408_babs_open_data/201408_trip_data.csv')
trip3 = pd.read_csv('/home/kyohei/bike_share_data/201508_trip_data.csv')
trip4 = pd.read_csv('/home/kyohei/bike_share_data/201608_trip_data.csv')

In [8]:
#trip1のcolumns名の変更
trip1.rename(columns={'Subscription Type':'Subscriber Type'},inplace=True)

In [9]:
#tripdataの結合
trip = pd.concat([trip1,trip2,trip3,trip4],axis=0,ignore_index=True)

In [27]:
#tripのスタートターミナルの限定
trip_hbp_start = trip[trip['Start Terminal']==50]

#datetimeに変換
trip_hbp_start['Start Date'] = trip_hbp_start['Start Date'].apply(lambda x:pd.to_datetime(x))

#年
trip_hbp_start['start_year'] = trip_hbp_start['Start Date'].apply(lambda x:x.date().year)

#月
trip_hbp_start['start_month'] = trip_hbp_start['Start Date'].apply(lambda x:x.date().month)

#曜日
trip_hbp_start['start_dayofweek'] = trip_hbp_start['Start Date'].apply(lambda x:x.dayofweek)

import datetime
#時刻
trip_hbp_start['start_time'] = trip_hbp_start['Start Date'].apply(lambda x:x.time())

/home/kyohei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kyohei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kyohei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [28]:
trip_hbp_start.head()

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,start_year,start_month,start_dayofweek,start_time
67,4748,240,2013-08-29 16:56:00,Harry Bridges Plaza (Ferry Building),50,8/29/2013 17:00,Beale at Market,56,626,Subscriber,94122,2013,8,3,16:56:00
76,4423,254,2013-08-29 12:50:00,Harry Bridges Plaza (Ferry Building),50,8/29/2013 12:54,Clay at Battery,41,544,Subscriber,94070,2013,8,3,12:50:00
110,4392,323,2013-08-29 12:34:00,Harry Bridges Plaza (Ferry Building),50,8/29/2013 12:39,Embarcadero at Vallejo,48,428,Subscriber,94123,2013,8,3,12:34:00
131,4570,351,2013-08-29 14:06:00,Harry Bridges Plaza (Ferry Building),50,8/29/2013 14:12,Washington at Kearney,46,275,Subscriber,94109,2013,8,3,14:06:00
132,4278,353,2013-08-29 11:52:00,Harry Bridges Plaza (Ferry Building),50,8/29/2013 11:58,Commercial at Montgomery,45,596,Subscriber,94123,2013,8,3,11:52:00


### weather

In [29]:
weather1 = pd.read_csv('/home/kyohei/bike_share_data/201402_babs_open_data/201402_weather_data.csv')
weather2 = pd.read_csv('/home/kyohei/bike_share_data/201408_babs_open_data/201408_weather_data.csv')
weather3 = pd.read_csv('/home/kyohei/bike_share_data/201508_weather_data.csv')
weather4 = pd.read_csv('/home/kyohei/bike_share_data/201608_weather_data.csv')

In [51]:
#columnsの修正
weather2.columns = weather2.columns.str.strip()
weather3.columns = weather3.columns.str.strip()
weather4.columns = weather4.columns.str.strip()
weather1.columns = weather2.columns
weather4.rename(columns={'ZIP':'Zip'},inplace=True)

In [53]:
#weatherの統合
weather = pd.concat([weather1,weather2,weather3,weather4],axis=0)

In [55]:
#dateの変換
weather['PDT'] = weather['PDT'].apply(lambda x:pd.to_datetime(x))

In [65]:
weather_SF = weather[weather.Zip==95113]

### status

In [66]:
status1 = pd.read_csv('/home/kyohei/bike_share/data_split_2013/San Francisco_station_status_stid50.csv')
status2 = pd.read_csv('/home/kyohei/bike_share/data_split_2014/San Francisco_station_status_stid50.csv')
status3 = pd.read_csv('/home/kyohei/bike_share/data_split_2015/San Francisco_station_status_stid50.csv')

In [67]:
status = pd.concat([status1,status2,status3],axis=0)

In [69]:
#datetime
status['time'] = status['time'].apply(lambda x:pd.to_datetime(x))

#時間情報
status['time_hour_minute'] = status['time'].apply(lambda x:datetime.time(x.time().hour,x.time().minute))

In [74]:
status = status.reset_index()

In [79]:
status.head()

,index,Unnamed: 0,station_id,bikes_available,docks_available,time,time_hour_minute
0,0,9809414,50,7,16,2013-08-29 12:06:01,12:06:00
1,1,9809415,50,7,16,2013-08-29 12:07:01,12:07:00
2,2,9809416,50,7,16,2013-08-29 12:08:01,12:08:00
3,3,9809417,50,7,16,2013-08-29 12:09:01,12:09:00
4,4,9809418,50,7,16,2013-08-29 12:10:01,12:10:00


In [88]:
status['time'] = status['time'].apply(lambda x:datetime.datetime(x.year,x.month,x.day,x.hour,x.minute))

### 統合前のデータ準備

In [99]:
#目的変数の作成
target = pd.DataFrame(trip_hbp_start.groupby('Start Date').size()).reset_index()

#columnの変換
target.rename(columns={0:'Count'},inplace=True)

In [110]:
#weather
T_mapping = {'T':0}
f = lambda x:T_mapping.get(x,x)
weather_SF['PrecipitationIn'] = weather_SF.PrecipitationIn.map(f).astype(float)

/home/kyohei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [116]:
#weather
event_dummy = pd.get_dummies(weather_SF.Events)

weather_SF_mod = pd.concat([weather_SF,event_dummy],axis=1)

weather_SF_sub = weather_SF_mod.drop(['Events','Max Gust SpeedMPH','Zip'],axis=1)

#前日データ
weather_SF_sub_yesterday = pd.concat([weather_SF_sub[['PDT']],weather_SF_sub.shift(1).drop(['PDT'],axis=1)],axis=1).dropna()

In [139]:
#Status
status_sub = status[['bikes_available','docks_available','time','time_hour_minute']]

#年
status_sub['year'] = status_sub['time'].apply(lambda x:x.year)

#月
status_sub['month'] = status_sub['time'].apply(lambda x:x.month)

#曜日
status_sub['dayofweek'] = status_sub['time'].apply(lambda x:x.dayofweek)

/home/kyohei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kyohei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [144]:
#dummy変数づくり
#年
dummy_year = pd.get_dummies(status_sub.year)

#時刻
dummy_time = pd.get_dummies(status_sub.time_hour_minute)

#月
dummy_month = pd.get_dummies(status_sub.month)

#曜日
weekday = {0:'monday',1:'tuseday',2:'wednseday',3:'thursday',4:'friday',5:'saturday',6:'sunday'}
status_sub['dayofweek'] = status_sub['dayofweek'].map(weekday)
dummy_weekday = pd.get_dummies(status_sub.dayofweek)

In [145]:
status_dummies = pd.concat[[dummy_year,dummy_time,dummy_month,dummy_weekday],axis=0]

,00:00:00,00:01:00,00:02:00,00:03:00,00:04:00,00:05:00,00:06:00,00:07:00,00:08:00,00:09:00,...,23:50:00,23:51:00,23:52:00,23:53:00,23:54:00,23:55:00,23:56:00,23:57:00,23:58:00,23:59:00
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
weekday = {0:'monday',1:'tuseday',2:'wednseday',3:'thursday',4:'friday',5:'saturday',6:'sunday'}
status_sub['dayofweek'].map(weekday).unique()


array(['thursday', 'friday', 'saturday', 'sunday', 'monday', 'tuseday',
       'wednseday'], dtype=object)